In [1]:
import pandas as pd

In [13]:
df = pd.read_csv('jeju.csv',encoding='cp949')
df.head(1)

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자,Unnamed: 8
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15 (일도일동),인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20,NaN


In [4]:
# 1. 주소 정보를 이용해서 위도,경도 값을 저장합니다. '위치' 컬럼에 (위도,경도) 저장
# 주소를 찾아서 저장하는 함수를 작성하여 처리  
# 함수 이름은 geo_coder(query) 리턴값 : (위도,경도)  만일 못찾았다면 '결과없음' 으로 리턴
# 함수사용, apply()를 이용해 데이터 프레임에 적용

In [17]:
def geo_coder(query):
    import requests
    REST_API_KEY = 'key'
    query = query.split('(')[0]
    url = f"https://dapi.kakao.com/v2/local/search/keyword.json?category_group_code=PO3&query={query}"
    headers = {'Authorization':f'KakaoAK {REST_API_KEY}'}
    res = requests.get(url,headers=headers)
    jsondata = res.json()
    # print(query)
    # print(jsondata)
    try:
        return jsondata['documents'][0]['y'],jsondata['documents'][0]['x']
    except:
        return '결과없음'

In [18]:
df['위치'] = df['주소'].apply(geo_coder)

In [19]:
df.head(1)

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자,Unnamed: 8,위치
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15 (일도일동),인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20,NaN,"(33.5150737361162, 126.526342790758)"


In [5]:
# 2. '위치' 컬럼의 값을 Marker 위치값으로, tooltip으로 읍면동을 사용하여 지도에 표시(클리스터이용)
# 함수사용, apply()를 이용해 데이터 프레임에 적용

In [ ]:
# 3. '일반현황'컬럼의 인구수만 분리하여 '인구수'라는 int형 컬럼을 생성